In [1]:
import psycopg2
from decouple import config

!. ../.env

conn = psycopg2.connect("postgres://" + config("POSTGRES_USERNAME") + ":" + config("POSTGRES_PASSWORD") + "@raja.db.elephantsql.com:5432/mozfsrjp")
curs = conn.cursor()

In [2]:
curs.execute("""
    DROP TABLE comments2;
""")

In [3]:
curs.execute("""
    CREATE TABLE comments2 (
        id BIGINT PRIMARY KEY,
        author VARCHAR(100),
        time BIGINT,
        comment_text TEXT,
        parent_id BIGINT,
        saltiness FLOAT
    );
""")

In [4]:
curs.close()
conn.commit()

### Load the CSV

In [5]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def escape_string(text):
    if isinstance(text, str):
        text = re.sub(r"\"", "\\\"", text)
        text = re.sub(r"'", "\\'", text)
        return text
    else:
        return "-"

def convert_int(x):
    try:
        return int(x)
    except:
        return -1

def get_saltiness(x):
    if isinstance(x, str):
        res = analyzer.polarity_scores(x)["neg"]
        return res
    return 0.0

In [6]:
def refine(df):
    
    df = df.loc[df['type'] == 'comment']
    df = df.loc[df['author'].isnull() == False]
    df = df[['id', 'time', 'author', 'parent', 'text', 'type']]
    df['parent'] = df['parent'].astype(int)    
    return df

def scrub(doc):
    patt = {
            "unicode_patt": "&.{4}(?=;);",
            "line_break":   "<p>",
            "href_patt":    "<a.*</a>",
            "quote":        "&quot;",
            "html_footnote": "\[.\]",
            "punctuation":   "[^\w\s]",
            "numbers":       "[^A-Za-z\s]",
        }
    
    r = rf'|'.join(patt.values())
    return re.sub(r, ' ', str(doc))

def process_text(df):
    
    # regex
    df['processed_text'] = df['text'].apply(scrub)
    # lowercase
    df['processed_text'] = df['processed_text'].str.lower()
    # double spaces
    df['processed_text'] = df['processed_text'].str.replace(r'\s+', ' ')
    
    
    # word freq
    word_freq = pd.Series(' '.join(df['processed_text']).split()).value_counts()
    
    common = list(word_freq[:10].index)
    rare = list(word_freq[word_freq.values < 2].index)
    
    stop_words = list(nltk.corpus.stopwords.words('english'))    
    stop_words = set(stop_words + common + rare)
    
    print('removing stopwords')
    pat = r'\b(?:{})\b'.format('|'.join(stop_words))
    
    df['no_stopwords'] = df['processed_text'].str.replace(pat, '')
    df['no_stopwords'] = df['no_stopwords'].str.replace(r'\s+', ' ')
    
    # remove less than 2 words
    df = df[df["no_stopwords"].str.split(" ").apply(lambda x: len(x)) > 2]
    
    return df

In [7]:
import pandas as pd

hn_df = pd.read_csv("../csv/most_recent_1_5mm.csv")

In [7]:
top_1000 = hn_df["author"].value_counts().index[:1000]
hn_df = hn_df[hn_df["author"].isin(top_1000)]

In [ ]:
from psycopg2.extras import execute_batch
import numpy as np
import nltk

batchsize = 10000

for ix in range(520000, len(hn_df)+1, batchsize):
    
    print(f"Batch {ix} / {len(hn_df)} -- {ix/len(hn_df)*100:.2f}%")
    
    batch = hn_df[ix:ix+batchsize]
    batch = process_text(batch)
    
    batch = [
        [
            row[1][1],
            row[1][2],
            row[1][3],
            row[1][4],
            convert_int(row[1][7]),
            get_saltiness(row[1][4]),
        ]
        for row in batch.iterrows()
    ]
    
    query = """
        INSERT INTO comments2 (id, author, time, comment_text, parent_id, saltiness)
        VALUES (%s, %s, %s, %s, %s, %s);
    """
    
    curs = conn.cursor()
    execute_batch(curs, query, batch)
    curs.close()

Batch 520000 / 1499356 -- 34.68%


/home/cowley/venv/lambda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/cowley/venv/lambda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/cowley/venv/lambda/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

removing stopwords


/home/cowley/venv/lambda/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/cowley/venv/lambda/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Batch 530000 / 1499356 -- 35.35%
removing stopwords
Batch 540000 / 1499356 -- 36.02%
removing stopwords
Batch 550000 / 1499356 -- 36.68%
removing stopwords
Batch 560000 / 1499356 -- 37.35%
removing stopwords
Batch 570000 / 1499356 -- 38.02%
removing stopwords
Batch 580000 / 1499356 -- 38.68%
removing stopwords
Batch 590000 / 1499356 -- 39.35%
removing stopwords
Batch 600000 / 1499356 -- 40.02%
removing stopwords
Batch 610000 / 1499356 -- 40.68%
removing stopwords
Batch 620000 / 1499356 -- 41.35%
removing stopwords
Batch 630000 / 1499356 -- 42.02%
removing stopwords
Batch 640000 / 1499356 -- 42.68%
removing stopwords
Batch 650000 / 1499356 -- 43.35%
removing stopwords
Batch 660000 / 1499356 -- 44.02%
removing stopwords
Batch 670000 / 1499356 -- 44.69%
removing stopwords
Batch 680000 / 1499356 -- 45.35%
removing stopwords
Batch 690000 / 1499356 -- 46.02%
removing stopwords
Batch 700000 / 1499356 -- 46.69%
removing stopwords
Batch 710000 / 1499356 -- 47.35%
removing stopwords
Batch 720000

In [ ]:
conn.commit()

In [ ]:
query = """
    SELECT *
    FROM comments
    LIMIT 1000
"""
curs = conn.cursor()
curs.execute(query)
res = curs.fetchall()

In [ ]:
res[0]

In [59]:
res[3]

(22310905,
 'apta',
 1581529723,
 'Ah yes, the alcohol excuse. Who would have thought that allowing alcohol on flights wasn&#x27;t the best idea?',
 22309335,
 0.0)

In [28]:
from datetime import datetime, timedelta

hn_df["time_dt"] = hn_df["time"].apply(datetime.utcfromtimestamp)

In [32]:
three_months_ago = (datetime.utcnow() - timedelta(90)).timestamp()

In [35]:
query = """
    DELETE FROM comments
    WHERE comments.time < %s;
"""
curs = conn.cursor()
curs.execute(query, [three_months_ago])
conn.commit()

In [36]:
curs = conn.cursor()
curs.execute("""
    SELECT COUNT(*)
    FROM comments;
""")
curs.fetchone()

(754093,)

In [55]:
query = """
    SELECT *
    FROM comments
    WHERE LENGTH(comment_text) > 3
    ORDER BY saltiness DESC
    LIMIT 100
"""
curs = conn.cursor()
curs.execute(query)
res = curs.fetchall()

In [56]:
res

[(22295248, 'jpseawell', 1581385590, 'Scary shit.', 22294841, 1.0),
 (22407657, 'thatdrew', 1582573583, 'wrong.', 22393828, 1.0),
 (21915733, 'hyperbovine', 1577732100, 'Demand.', 21907811, 1.0),
 (22118709, 'Accujack', 1579709246, 'Spam.', 22116657, 1.0),
 (22113584, 'pedramnegahdar', 1579655578, 'Pathetic.', 22106536, 1.0),
 (22011882, 'Melkor765', 1578672239, 'Selfish', 22008441, 1.0),
 (21734037, 'wnmurphy', 1575772629, 'Seriously.', 21733689, 1.0),
 (22091749, 'brandchirps', 1579446106, 'no, sorry.', 22089473, 1.0),
 (22384612, 'the_other', 1582304286, 'STOP!', 22384286, 1.0),
 (22200449, 'LatteLazy', 1580465339, 'SPAM', 22200400, 1.0),
 (22422156, 'zulgan', 1582717326, 'rebel.', 22422133, 1.0),
 (21990162, 'Pusha_Drugz', 1578479615, 'sad :(', 21989961, 1.0),
 (22126501, 'badrabbit', 1579777158, 'Misinformation', 22125277, 1.0),
 (21711215, 'bayesian_horse', 1575540843, 'Depressing.', 21710564, 1.0),
 (21751194, 'googlycooly', 1575972799, 'SPAM!', 21751156, 1.0),
 (22154160, 'star